## Author:  Matthias Bartolo               Id: 0436103L

# Course Project
# ICS2207 - Machine Learning: Classification, Search and Optimisation

### Packages

In [1]:
!pip install pillow 
import random
import re
import math
import matplotlib.pyplot as plt
import time
from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont

^C


### Function which Loads Dataset (text corpus) , given a filename

#### The text corpus used was the following: Ben-Hur: A tale of the Christ by Lew Wallace
obtained from: https://www.gutenberg.org/ebooks/2145

In [2]:
def LoadingAndFilteringDataset(filename):
    #Opening dataset file
    dataset= open(filename, mode='r',encoding="utf8")
    #Reading from file
    unfilteredtext=dataset.read()
    #Filtering the characters in the dataset, with the characters present in the keyboard
    filteredtext=re.sub('[^A-Za-z.,;\" ]',"",unfilteredtext)
    #Transforming all text to lower case
    filteredtext=filteredtext.lower()
    #Closing dataset file
    dataset.close()
    #Returning filteredText
    return filteredtext

In [3]:
#Loading and Filtering Dataset (Variable will be used continously)
dataset=LoadingAndFilteringDataset("dataset.txt")

### Function which Loads a Keyboard Image, and prints the keys on the keyboard

#### The following resources were used, for the facilitation of creation of such function:
Keyboard Image retrieved from: https://timvandevall.com/free-blank-keyboard-template-printable/

Utilising font from: https://fonts.google.com/specimen/Roboto

In [4]:
def ShowKeyboard(keys):
    #Opening keyboard image
    img = Image.open('KeyboardImage.png')
    
    #Calling draw Method to add 2D graphics to the image
    textOnImage = ImageDraw.Draw(img)
    #Setting font
    keyboardFont = ImageFont.truetype('Roboto-Black.ttf', 135)
 
    #Adding text to keyboard Image
    #Drawing text with offsets, offsets have been calculated through trial and error
    startingx=400
    startingy=1050
    xOffset=220
    yOffset=0
    multiplyrate=0
    #Looping through all the keys and adding the keys to the keyboard image, based on calculated offset
    for i in range(0,len(keys)):
        if(i%10==0 and i!=0):
            yOffset+=230+i
            startingx+=5*i
            multiplyrate=0
        textOnImage.text((startingx+xOffset*multiplyrate, startingy+yOffset), keys[i], font=keyboardFont)
        multiplyrate+=1

    #Returning image
    return img

In [5]:
#Variable to hold keys:
#Qwerty keys
keys=[ "q", "w", "e", "r", "t", "y", "u", "i", "o", "p", "a", "s", "d", "f", "g", "h", "j", "k", "l","\"", "z", "x", "c", "v", "b", "n", "m","." ,",", ";" ]

### Function which returns  a Dictionary of weight positions given a word index

In [6]:
def AssignWeights(keys):
    #Utilising same offsets as ShowKeyboard() method
    weights=[]
    startingx=400
    startingy=1050
    xOffset=220
    yOffset=0
    multiplyrate=0
    #Looping through all keys, and appending to the weights list, the x and y calculated positions of the keys on the keyboard
    for i in range(0,len(keys)):
        if(i%10==0 and i!=0):
            yOffset+=230+i
            startingx+=5*i
            multiplyrate=0
        weights.append([startingx+xOffset*multiplyrate, startingy+yOffset])
        multiplyrate+=1
    
    #Returning a dictionary, from the wieghts list, to ensure fast searches
    return {i:weights[i] for i in range(0,len(weights))}

In [7]:
#Assigning weights (Variable will be used continously)
weights=AssignWeights(keys)

### Function which calculates, the distance, for a character entered, and given the current finger positions on the keyboard 

In [8]:
#Function parameters include:
#1. fingerpointer - to hold the position of the current finger on the keyboard
#2. characterpointer - to hold the position of the next character to type on the keyboard
#3. weights - the dictionary of weights for all the keys
def KeyboardCharacterDistance(fingerpointer,characterpointer,weights):
    distance=0
    #If fingerpointer is characterpointer, i.e., current finger is on current character, then the distance returned is 0
    if fingerpointer==characterpointer:
        return distance
    #The function will attempt to work out the euclidean distance, between the current position of the finger (fingerpointer)
    #and, the position of the character entered (characterpointer).
    #The function will the use the weight dictionary, to find the x and y positions of the two pointers, and
    #calculate, the euclidean distance on said values
    adjacent=abs(weights[fingerpointer][0]-weights[characterpointer][0])
    opposite=abs(weights[fingerpointer][1]-weights[characterpointer][1])
    hyphotenuse=math.sqrt(pow(adjacent,2)+pow(opposite,2))
    #Returning rounded euclidean distance
    return round(hyphotenuse/220,3)

### Function which calculates, the Fitness Function, of a given keyboard, and given text corpus

In [9]:
#Function parameters include:
#1. keyboard - variable to hold keyboard
#2. weights - the dictionary of weights for all the keys
#3. text - the string text corpus
def FitnessFunction(keyboard,weights, text):
    #Initialising totalDistance and fingerpointers
    totalDistance=0
    distance=0
    fingerpointers=[10,11,12,13,16,17,18,19]
    
    #Looping through all the characters in text 
    for currentChar in text:
        #Only entering if statement if current character is not a space
        if currentChar!=" ":
            #Retrieving index of current character in keyboard
            charIndex=keyboard.index(currentChar)
            #Calculating fingerpointer index based on current character index
            fingerpointerIndex=charIndex%10
            #Setting respective fingerpointers, since there is a partition in the middle of the keyboard, where there
            #are no fingers
            if fingerpointerIndex>=4 and fingerpointerIndex<6:
                fingerpointerIndex-=1
            elif fingerpointerIndex>=6:
                fingerpointerIndex-=2
            #Calculating distance via KeyboardCharacterDistance function, for current character
            distance=KeyboardCharacterDistance(fingerpointers[fingerpointerIndex],charIndex,weights)
            #Adding distance to totalDistance
            totalDistance+=distance
            #Resetting all fingerpointers, except the last move 
            fingerpointers=[10,11,12,13,16,17,18,19]
            fingerpointers[fingerpointerIndex]=charIndex
    
    #Returning current keyboard and rounded total distance
    return [keyboard,round(totalDistance,3)]

### Function which performs Single Point Crossover, given 2 parents

In [10]:
#Function parameters include:
#1. parent1 - variable to hold keyboard parent1
#2. parent2 - variable to hold keyboard parent2
def SinglePointCrossover(parent1, parent2):
    #Randomly selecting a splitting point between parents
    splittingPoint=random.randint(1,(len(parent1)-1))
    #Declaring children to be returned
    child1=[]
    child2=[]
    
    #Declaring and Initialising indexes to 0
    index1=0
    index2=0
    
    #While index1 is smaller than the splitting point, then add parent1 to child and incrementing index1
    while(index1<splittingPoint):
        if parent1[index1] not in child1:
            child1.append(parent1[index1])
            index1+=1
    #While index2 is smaller than the splitting point, then add parent2 to child and incrementing index2
    while(index2<splittingPoint):
        if parent2[index2] not in child2:
            child2.append(parent2[index2])
            index2+=1
    
    #Resetting indexes
    pos=index1
    errorOffset=0
    #While index1 is smaller than parent2, then continue to add keys from parent2, if keys are not already present in child
    #If keys are already present in child, then proceed to find a key from the first half of parent2, which child does not have,
    #and proceed to add said key
    while(index1<len(parent2)):
        if parent2[pos] not in child1:
            child1.append(parent2[pos])
            index1+=1
            pos=index1
        else:
            errorOffset+=1
            pos=index1-errorOffset
    #Resetting indexes
    pos=index2
    errorOffset=0
    #While index2 is smaller than parent1, then continue to add keys from parent1 if keys are not already present in child
    #If keys are already present in child, then proceed to find a key from the first half of parent1, which child does not have,
    #and proceed to add said key
    while(index2<len(parent1)):
        if parent1[pos] not in child2:
            child2.append(parent1[pos])
            index2+=1
            pos=index2
        else:
            errorOffset+=1
            pos=index2-errorOffset
    
    #Returning children
    return child1, child2

### Function which performs Double Point Crossover, given 2 parents

In [11]:
#Function parameters include:
#1. parent1 - variable to hold keyboard parent1
#2. parent2 - variable to hold keyboard parent2
def DoublePointCrossover(parent1, parent2):
    #Randomly selecting first splitting point between parents
    splittingPoint1=random.randint(1,round(len(parent1)/2))
    #Randomly selecting second splitting point between parents
    splittingPoint2=random.randint(splittingPoint1,(len(parent1)-1))
    #Declaring children to be returned
    child1=[]
    child2=[]
    
    #Declaring and Initialising indexes to 0
    index1=0
    index2=0
    #While index1 is smaller than first splitting point, then add parent1 to child and incrementing index1
    while(index1<splittingPoint1):
        if parent1[index1] not in child1:
            child1.append(parent1[index1])
            index1+=1
    #While index2 is smaller than first splitting point, then add parent2 to child and incrementing index2
    while(index2<splittingPoint1):
        if parent2[index2] not in child2:
            child2.append(parent2[index2])
            index2+=1
    
    #Resetting indexes
    pos=index1
    errorOffset=0
    #While index1 is smaller than second splitting point, then continue to add keys from parent2 if keys are not already present in child
    #If keys are already present in child, then proceed to find a key from the first half of parent2, which child does not have,
    #and proceed to add said key
    while(index1<splittingPoint2):
        if parent2[pos] not in child1:
            child1.append(parent2[pos])
            index1+=1
            pos=index1
        else:
            errorOffset+=1
            pos=index1-errorOffset
    #Resetting indexes
    pos=index2
    errorOffset=0
    #While index2 is smaller than second splitting point, then continue to add keys from parent1 if keys are not already present in child
    #If keys are already present in child, then proceed to find a key from the first half of parent1, which child does not have,
    #and proceed to add said key
    while(index2<splittingPoint2):
        if parent1[pos] not in child2:
            child2.append(parent1[pos])
            index2+=1
            pos=index2
        else:
            errorOffset+=1
            pos=index2-errorOffset
    #Resetting indexes
    pos=index1
    errorOffset=0
    #While index1 is smaller than parent1, then continue to add keys from parent1 if keys are not already present in child
    #If keys are already present in child, then proceed to find a key from the first half of parent1, which child does not have,
    #and proceed to add said key
    while(index1<len(parent1)):
        if parent1[pos] not in child1:
            child1.append(parent1[pos])
            index1+=1
            pos=index1
        else:
            errorOffset+=1
            pos=index1-errorOffset
    #Resetting indexes
    pos=index2
    errorOffset=0
    #While index2 is smaller than parent2, then continue to add keys from parent2 if keys are not already present in child
    #If keys are already present in child, then proceed to find a key from the first half of parent2, which child does not have,
    #and proceed to add said key
    while(index2<len(parent2)):
        if parent2[pos] not in child2:
            child2.append(parent2[pos])
            index2+=1
            pos=index2
        else:
            errorOffset+=1
            pos=index2-errorOffset
    
    #Returning children            
    return child1, child2

### Mutation Functions:

Different mutations inspired from: https://www.tutorialspoint.com/genetic_algorithms/genetic_algorithms_mutation.htm

#### Function which performs Swap Mutation given a child

In [12]:
def SwapMutation(child):
    #Generating 2 random indexes based on list length
    randindex1=random.randint(0,len(child)-1)
    randindex2=random.randint(0,len(child)-1)
    #Making sure that indexes are not the same
    while randindex1==randindex2:
        randindex1=random.randint(0,len(child)-1)
    #Swapping keys at both random indexes
    child[randindex1],child[randindex2]=child[randindex2],child[randindex1]

#### Function which performs Scramble Mutation given a child

In [13]:
def ScrambleMutation(child):
    #Generating a random index
    randindex1=random.randint(0,round(len(child)/2))
    #Generating a secondary index, based on randindex1
    randindex2=random.randint(randindex1+1,round(len(child)-1))
    
    #Setting reverseindex to randindex2 and index to randindex1
    reverseindex=randindex2
    index=randindex1
    #Generating swapindex based on index and reverseindex
    swapindex=random.randint(index,reverseindex)
    #Looping until index is smaller than reverseindex
    while(index<reverseindex):
        #Generating swapindex based on index and reverseindex
        swapindex=random.randint(index+1,reverseindex)
        #Swapping keys of positions which have indexes: index and swapindex
        child[index],child[swapindex]=child[swapindex],child[index]
        #Incrementing index and decrementing reverseindex
        index+=1
        reverseindex-=1

#### Function which performs Inversion Mutation given a child

In [14]:
def InversionMutation(child):
    #Generating a random index
    randindex1=random.randint(0,round(len(child)/2))
    #Generating a secondary index, based on randindex1
    randindex2=random.randint(randindex1+1,round(len(child)-1))
    
    #Setting reverseindex to randindex2 and index to randindex1
    reverseindex=randindex2
    index=randindex1
    #Looping until index is smaller than reverseindex
    while(index<reverseindex):
        #Swapping keys of positions which have indexes: index and reverseindex
        child[index],child[reverseindex]=child[reverseindex],child[index]
        #Incrementing index and decrementing reverseindex
        index+=1
        reverseindex-=1

### Evaluation Functions:

#### Function which given a population, outputs Evaluation Metrics:
##### - Best Population Fitness
##### - Worst Population Fitness
##### - Average Population Fitness

In [15]:
def GetEvaluationMetrics(population):
    #Declaring totalFitness and averagePopulationFitness
    totalFitness=0
    averagePopulationFitness=0
    #Looping through all the population and appending the fitness of a keyboard to the totalfitness
    for p in population:
        totalFitness+=p[1]
    #Acquiring averagePopulationFitness
    averagePopulationFitness=totalFitness/len(population)
    #Returning respective metrics
    return population[0][1],population[len(population)-1][1],averagePopulationFitness

#### Function which Draws a Line Graph, given Population values array, generation array and ylabel

In [16]:
def DrawGraph(worstFitnessArray,bestFitnessArray,avgFitnessArray,generation):
    #Plotting the points with labels
    plt.plot(generation, worstFitnessArray, label = "Worst Population Fitness")
    plt.plot(generation, bestFitnessArray, label = "Best Population Fitness")
    plt.plot(generation, avgFitnessArray, label = "Avg Population Fitness")

    #Naming the x-axis
    plt.xlabel("Generation")
    #Naming the y-axis
    plt.ylabel("Fitness")

    #Giving a title to the graph
    plt.title("Fitness vs Generation")
    #Displaying legend
    plt.legend(loc='upper right')
    #Displaying plot
    plt.show()

### The following function will be used by the Main Genetic Algorithm Function:

#### Function which creates the initial generation, given a population size, population array, dataset and keyboard keys

In [17]:
def CreateFirstGeneration(populationSize,population,dataset,keys,weights):
    #Looping according to the populationsize
    for num in range(populationSize):

        #Randomly shuffling the keys
        random.shuffle(keys)

        #Creating a keyboard from the shuffled keys
        keyboard=keys.copy()

        #Appending keyboard to the population and calculating the keyboard's Fitness
        population.append(FitnessFunction(keyboard,weights, dataset))


#### Functions which are used to sort the population array

In [18]:
def SortPopulationByFitness(population):
    #Sorting population
    population.sort(key=GetSortKey)
       
def GetSortKey(population):
    return population[1] 

### Main Genetic Algorithm Function

#### Function outputs the best keyboard, after a number of generations

#### Function takes the following parameters:
##### 1. keys - variable to hold the keys which will be used to create keyboards
##### 2. dataset - variable to hold the text corpus as a string
##### 3. weights - variable to hold the dictionary of weights for all the keys
##### 4. populationSize - variable to hold the size of the population
##### 5. noOfGenerations - variable to hold the amount of generations
##### 6. crossoverBit - variable which will be used to determine which crossover operation to perform
##### 7. mutationBit - variable which will be used to determine which mutation operation to perform
##### 8. mutationRate - variable which will be used to determine the rate of mutation operation
##### 9. crossoverRate - variable which will be used to determine the rate of the crossover operation
##### 10. elitismRate - variable which will be used to determine the rate of the elitism operation

In [19]:
def GeneticAlgorithmMain(keys,dataset,weights,populationsize,noOfGenerations,crossoverBit,mutationBit,mutationRate,crossoverRate,elitismRate):
    #Declaring lists, which will be used for evaluation and to hold best keyboard
    bestvalue=[]
    worstvalue=[]
    averagevalue=[]
    generationvalue=[]
    bestkeyboard=[]
    
    #Declaring population array
    population=[]
    #Creation of first generation by invoking CreateFirstGeneration Method
    CreateFirstGeneration(populationsize,population,dataset,keys,weights)
    #Looping for a number of generations
    for generation in range(noOfGenerations):
        #Sorting the current population
        SortPopulationByFitness(population)
        #Declaring new population list to hold the new population
        newpopulation=[]
        
        #Elitism:
        #Calculating number of elites to append to the new population
        sizeOfElitism=round(populationsize*elitismRate)
        #Appending elites to the new population
        newpopulation.extend(population[0:sizeOfElitism])
        

        #Variable to hold saved index of parent 1
        pIndex=0
        index=0
        #Looping until newpopulation size is equal to population size
        while len(population)>len(newpopulation):
                #Setting Parent1Index to pIndex
                Parent1Index= pIndex
                #Setting Parent2Index to Parent1Index+a random offset between 0 and length of half of the population
                Parent2Index= Parent1Index+random.randint(0,round(len(population)/2))
                #Generating Crossover Chance from a range of 0 and 1, and rounding to 1 decimalpoint
                CrossoverChance=round(random.uniform(0,1),1)
               
                #Crossover:
                #If CrossoverChance is smaller than CrossoverRate, then attempt to perform Crossover
                if CrossoverChance< crossoverRate:
                    #Based on CrossoverBit, respective Crossover Method will be called
                    if crossoverBit==2:
                        children=DoublePointCrossover(population[Parent1Index][0],population[Parent2Index][0])
                    else:
                        children=SinglePointCrossover(population[Parent1Index][0],population[Parent2Index][0])
                    #Incrementing pIndex
                    pIndex+=1
                #Else no crossover is performed, and the children taken would be the parents
                else:
                    children=[population[Parent1Index][0],population[Parent2Index][0]]
                    
                #Mutation:
                #Generating Child1 Mutation Chance from a range of 0 and 1, and rounding to 1 decimalpoint
                Child1MutationChance=round(random.uniform(0,1),1)
                #Generating Child2 Mutation Chance from a range of 0 and 1, and rounding to 1 decimalpoint
                Child2MutationChance=round(random.uniform(0,1),1)
                #Based on mutationBit, respective Mutation Method will be called
                #For every child, if the mutation Chance, is smaller than, the mutation rate, then attempt
                #to perform mutation, else no mutation is performed
                if mutationBit==1:
                    if Child1MutationChance < mutationRate:
                        SwapMutation(children[0])
                    if Child2MutationChance < mutationRate:
                        SwapMutation(children[1])
                elif mutationBit==2:
                    if Child1MutationChance < mutationRate:
                        ScrambleMutation(children[0])
                    if Child2MutationChance < mutationRate:
                        ScrambleMutation(children[1])
                elif mutationBit==3: 
                    if Child1MutationChance < mutationRate:
                        InversionMutation(children[0])
                    if Child2MutationChance < mutationRate:
                        InversionMutation(children[1])
                
                #Calculating Fitness of new children, and appending entities to the new population
                newpopulation.append(FitnessFunction(children[0],weights, dataset))
                if len(population)>len(newpopulation):
                    newpopulation.append(FitnessFunction(children[1],weights, dataset))
                
        #Setting the population to the new population        
        population=newpopulation.copy()
        
        #Sorting the population
        SortPopulationByFitness(population)
        #Printing generation number and Best Population Fitness
        print("Generation: ",(generation+1)," Best Population Fitness",population[0])
        print()
        #Retrieving Evaluation metrics of population and appending them to relevant lists
        best,worst,average=GetEvaluationMetrics(population)
        bestvalue.append(best)
        worstvalue.append(worst)
        averagevalue.append(average)
        generationvalue.append(generation)
        #Setting best keyboard variable
        bestkeyboard=population[0]
    
    #Drawing appropriate Graphs
    DrawGraph(worstvalue,bestvalue,averagevalue,generationvalue)
    #Returning best keyboard
    return bestkeyboard
       

### QWERTY and AZERTY Keyboard Layouts Fitness results

In [20]:
#Variable which holds qwerty Keyboard
qwertyKeyboard=[ "q", "w", "e", "r", "t", "y", "u", "i", "o", "p", "a", "s", "d", "f", "g", "h", "j", "k", "l",";", "z", "x", "c", "v", "b", "n", "m","," ,".", "\"" ]
#Calculating and printing Fitness
fitness1=FitnessFunction(qwertyKeyboard,weights, dataset)
print("The qwerty keyboard has the following fitness: ",fitness1[1])
print("The keyboard has a Fitness/Character of:",(fitness1[1]/len(dataset)))

The qwerty keyboard has the following fitness:  815030.085
The keyboard has a Fitness/Character of: 0.7537460117672518


In [22]:
#Variable which holds azerty Keyboard
azertyKeyboard=[ "a", "z", "e", "r", "t", "y", "u", "i", "o", "p", "q", "s", "d", "f", "g", "h", "j", "k", "l","m", "w", "x", "c", "v", "b", "n", ";","," ,".", "\"" ]
#Calculating and printing Fitness
fitness2=FitnessFunction(azertyKeyboard,weights, dataset)
print("The azerty keyboard has the following fitness: ",fitness2[1])
print("The keyboard has a Fitness/Character of:",(fitness2[1]/len(dataset)))

The azerty keyboard has the following fitness:  871862.803
The keyboard has a Fitness/Character of: 0.8063053409488156


### Test Cases:

#### Test Case 1: Testing with a population of 10, for 100 generations, Single Point Crossover, and a crossover rate of 0.9

In [ ]:
populationsize=10
noOfGenerations=100
crossoverBit=1
mutationBit=0
mutationRate=0
crossoverRate=0.9
elitismRate=0

start = time.time()
bestkeyboard1=GeneticAlgorithmMain(keys,dataset,weights,populationsize,noOfGenerations,crossoverBit,mutationBit,mutationRate,crossoverRate,elitismRate)
end = time.time()
print("Time taken to execute: ",round((end-start),2)," s")

In [ ]:
print("The keyboard has the following Fitness: ",bestkeyboard1[1])
print("The keyboard has a Fitness/Character of:",(bestkeyboard1[1]/len(dataset)))
ShowKeyboard(bestkeyboard1[0])

#### Test Case 2: Testing with a population of 100, for 100 generations, Single Point Crossover, and a crossover rate of 0.9

In [ ]:
populationsize=100
noOfGenerations=100
crossoverBit=1
mutationBit=0
mutationRate=0
crossoverRate=0.9
elitismRate=0

start = time.time()
bestkeyboard2=GeneticAlgorithmMain(keys,dataset,weights,populationsize,noOfGenerations,crossoverBit,mutationBit,mutationRate,crossoverRate,elitismRate)
end = time.time()
print("Time taken to execute: ",round((end-start),2)," s")

In [ ]:
print("The keyboard has the following Fitness: ",bestkeyboard2[1])
print("The keyboard has a Fitness/Character of:",(bestkeyboard2[1]/len(dataset)))
ShowKeyboard(bestkeyboard2[0])

#### Test Case 3: Testing with a population of 100, for 100 generations, Single Point Crossover, and a crossover rate of 0.4

In [ ]:
populationsize=100
noOfGenerations=100
crossoverBit=1
mutationBit=0
mutationRate=0
crossoverRate=0.4
elitismRate=0

start = time.time()
bestkeyboard3=GeneticAlgorithmMain(keys,dataset,weights,populationsize,noOfGenerations,crossoverBit,mutationBit,mutationRate,crossoverRate,elitismRate)
end = time.time()
print("Time taken to execute: ",round((end-start),2)," s")

In [ ]:
print("The keyboard has the following Fitness: ",bestkeyboard3[1])
print("The keyboard has a Fitness/Character of:",(bestkeyboard3[1]/len(dataset)))
ShowKeyboard(bestkeyboard3[0])

#### Test Case 4: Testing with a population of 100, for 100 generations, Double Point Crossover, and a crossover rate of 0.9

In [ ]:
populationsize=100
noOfGenerations=100
crossoverBit=2
mutationBit=0
mutationRate=0
crossoverRate=0.9
elitismRate=0

start = time.time()
bestkeyboard4=GeneticAlgorithmMain(keys,dataset,weights,populationsize,noOfGenerations,crossoverBit,mutationBit,mutationRate,crossoverRate,elitismRate)
end = time.time()
print("Time taken to execute: ",round((end-start),2)," s")

In [ ]:
print("The keyboard has the following Fitness: ",bestkeyboard4[1])
print("The keyboard has a Fitness/Character of:",(bestkeyboard4[1]/len(dataset)))
ShowKeyboard(bestkeyboard4[0])

#### Test Case 5: Testing with a population of 100, for 100 generations, Double Point Crossover, and a crossover rate of 0.9, Swap Mutation with mutation rate of 0.1

In [ ]:
populationsize=100
noOfGenerations=100
crossoverBit=2
mutationBit=1
mutationRate=0.1
crossoverRate=0.9
elitismRate=0

start = time.time()
bestkeyboard5=GeneticAlgorithmMain(keys,dataset,weights,populationsize,noOfGenerations,crossoverBit,mutationBit,mutationRate,crossoverRate,elitismRate)
end = time.time()
print("Time taken to execute: ",round((end-start),2)," s")

In [ ]:
print("The keyboard has the following Fitness: ",bestkeyboard5[1])
print("The keyboard has a Fitness/Character of:",(bestkeyboard5[1]/len(dataset)))
ShowKeyboard(bestkeyboard5[0])

#### Test Case 6: Testing with a population of 100, for 100 generations, Double Point Crossover, and a crossover rate of 0.9, Scramble Mutation with mutation rate of 0.1

In [ ]:
populationsize=100
noOfGenerations=100
crossoverBit=2
mutationBit=2
mutationRate=0.1
crossoverRate=0.9
elitismRate=0

start = time.time()
bestkeyboard6=GeneticAlgorithmMain(keys,dataset,weights,populationsize,noOfGenerations,crossoverBit,mutationBit,mutationRate,crossoverRate,elitismRate)
end = time.time()
print("Time taken to execute: ",round((end-start),2)," s")

In [ ]:
print("The keyboard has the following Fitness: ",bestkeyboard6[1])
print("The keyboard has a Fitness/Character of:",(bestkeyboard6[1]/len(dataset)))
ShowKeyboard(bestkeyboard6[0])

#### Test Case 7: Testing with a population of 100, for 100 generations, Double Point Crossover, and a crossover rate of 0.9, Inversion Mutation with mutation rate of 0.1

In [ ]:
populationsize=100
noOfGenerations=100
crossoverBit=2
mutationBit=3
mutationRate=0.1
crossoverRate=0.9
elitismRate=0

start = time.time()
bestkeyboard7=GeneticAlgorithmMain(keys,dataset,weights,populationsize,noOfGenerations,crossoverBit,mutationBit,mutationRate,crossoverRate,elitismRate)
end = time.time()
print("Time taken to execute: ",round((end-start),2)," s")

In [ ]:
print("The keyboard has the following Fitness: ",bestkeyboard7[1])
print("The keyboard has a Fitness/Character of:",(bestkeyboard7[1]/len(dataset)))
ShowKeyboard(bestkeyboard7[0])

#### Test Case 8: Testing with a population of 100, for 100 generations, Double Point Crossover, and a crossover rate of 0.9, Scramble Mutation with mutation rate of 0.1, and elitism rate of 0.1

In [ ]:
populationsize=100
noOfGenerations=100
crossoverBit=2
mutationBit=2
mutationRate=0.1
crossoverRate=0.9
elitismRate=0.1

start = time.time()
bestkeyboard8=GeneticAlgorithmMain(keys,dataset,weights,populationsize,noOfGenerations,crossoverBit,mutationBit,mutationRate,crossoverRate,elitismRate)
end = time.time()
print("Time taken to execute: ",round((end-start),2)," s")

In [ ]:
print("The keyboard has the following Fitness: ",bestkeyboard8[1])
print("The keyboard has a Fitness/Character of:",(bestkeyboard8[1]/len(dataset)))
ShowKeyboard(bestkeyboard8[0])

#### Test Case 9: Testing with a population of 100, for 100 generations, Double Point Crossover, and a crossover rate of 0.9, Inversion Mutation with mutation rate of 0.1, and elitism rate of 0.1

In [ ]:
populationsize=100
noOfGenerations=100
crossoverBit=2
mutationBit=3
mutationRate=0.1
crossoverRate=0.9
elitismRate=0.1

start = time.time()
bestkeyboard9=GeneticAlgorithmMain(keys,dataset,weights,populationsize,noOfGenerations,crossoverBit,mutationBit,mutationRate,crossoverRate,elitismRate)
end = time.time()
print("Time taken to execute: ",round((end-start),2)," s")

In [ ]:
print("The keyboard has the following Fitness: ",bestkeyboard9[1])
print("The keyboard has a Fitness/Character of:",(bestkeyboard9[1]/len(dataset)))
ShowKeyboard(bestkeyboard9[0])

#### Test Case 10: Testing with a population of 15, for 70 generations, Double Point Crossover, and a crossover rate of 0.9, Scramble Mutation with mutation rate of 0.2, and elitism rate of 0.1

In [ ]:
populationsize=15
noOfGenerations=70
crossoverBit=2
mutationBit=2
mutationRate=0.2
crossoverRate=0.9
elitismRate=0.1

start = time.time()
bestkeyboard10=GeneticAlgorithmMain(keys,dataset,weights,populationsize,noOfGenerations,crossoverBit,mutationBit,mutationRate,crossoverRate,elitismRate)
end = time.time()
print("Time taken to execute: ",round((end-start),2)," s")

In [ ]:
print("The keyboard has the following Fitness: ",bestkeyboard10[1])
print("The keyboard has a Fitness/Character of:",(bestkeyboard10[1]/len(dataset)))
ShowKeyboard(bestkeyboard10[0])

#### Test Case 11: Testing with a population of 15, for 70 generations, Double Point Crossover, and a crossover rate of 0.9, Inversion Mutation with mutation rate of 0.2, and elitism rate of 0.1

In [ ]:
populationsize=15
noOfGenerations=70
crossoverBit=2
mutationBit=3
mutationRate=0.2
crossoverRate=0.9
elitismRate=0.1

start = time.time()
bestkeyboard11=GeneticAlgorithmMain(keys,dataset,weights,populationsize,noOfGenerations,crossoverBit,mutationBit,mutationRate,crossoverRate,elitismRate)
end = time.time()
print("Time taken to execute: ",round((end-start),2)," s")

In [ ]:
print("The keyboard has the following Fitness: ",bestkeyboard11[1])
print("The keyboard has a Fitness/Character of:",(bestkeyboard11[1]/len(dataset)))
ShowKeyboard(bestkeyboard11[0])

#### Test Case 12: Testing with a population of 15, for 70 generations, Double Point Crossover, and a crossover rate of 0.9, Inversion Mutation with mutation rate of 0.2, and elitism rate of 0.3

In [ ]:
populationsize=15
noOfGenerations=70
crossoverBit=2
mutationBit=3
mutationRate=0.2
crossoverRate=0.9
elitismRate=0.3

start = time.time()
bestkeyboard12=GeneticAlgorithmMain(keys,dataset,weights,populationsize,noOfGenerations,crossoverBit,mutationBit,mutationRate,crossoverRate,elitismRate)
end = time.time()
print("Time taken to execute: ",round((end-start),2)," s")

In [ ]:
print("The keyboard has the following Fitness: ",bestkeyboard12[1])
print("The keyboard has a Fitness/Character of:",(bestkeyboard12[1]/len(dataset)))
ShowKeyboard(bestkeyboard12[0])

#### Test Case 13: Testing with a population of 15, for 70 generations, Double Point Crossover, and a crossover rate of 0.9, Swap Mutation with mutation rate of 0.2, and elitism rate of 0.3

In [ ]:
populationsize=15
noOfGenerations=70
crossoverBit=2
mutationBit=1
mutationRate=0.2
crossoverRate=0.9
elitismRate=0.3

start = time.time()
bestkeyboard13=GeneticAlgorithmMain(keys,dataset,weights,populationsize,noOfGenerations,crossoverBit,mutationBit,mutationRate,crossoverRate,elitismRate)
end = time.time()
print("Time taken to execute: ",round((end-start),2)," s")

In [ ]:
print("The keyboard has the following Fitness: ",bestkeyboard13[1])
print("The keyboard has a Fitness/Character of:",(bestkeyboard13[1]/len(dataset)))
ShowKeyboard(bestkeyboard13[0])

### Testing Most Optimal Keyboard with other dataset
### The text corpus used was the following: The Story of Malta by Maturin M. Ballou
obtained from: https://www.gutenberg.org/ebooks/34036

In [ ]:
#Loading and Filtering Dataset (Variable will be used continously)
dataset2=LoadingAndFilteringDataset("dataset2.txt")

### QWERTY and AZERTY Keyboard Layouts Fitness results for new dataset

In [ ]:
#Variable which holds qwerty Keyboard
qwertyKeyboard=[ "q", "w", "e", "r", "t", "y", "u", "i", "o", "p", "a", "s", "d", "f", "g", "h", "j", "k", "l",";", "z", "x", "c", "v", "b", "n", "m","," ,".", "\"" ]
#Calculating and printing Fitness
fitness1=FitnessFunction(qwertyKeyboard,weights, dataset2)
print("The qwerty keyboard has the following fitness: ",fitness1[1])
print("The keyboard has a Fitness/Character of:",(fitness1[1]/len(dataset2)))

In [ ]:
#Variable which holds azerty Keyboard
azertyKeyboard=[ "a", "z", "e", "r", "t", "y", "u", "i", "o", "p", "q", "s", "d", "f", "g", "h", "j", "k", "l","m", "w", "x", "c", "v", "b", "n", ";","," ,".", "\"" ]
#Calculating and printing Fitness
fitness2=FitnessFunction(azertyKeyboard,weights, dataset2)
print("The azerty keyboard has the following fitness: ",fitness2[1])
print("The keyboard has a Fitness/Character of:",(fitness2[1]/len(dataset2)))

### Testing Most Optimal Keyboard with new dataset

In [ ]:
#Variable which holds optimal Keyboard
optimalKeyboard=[ "d", "f", "y", "g", "p", "x", "l", "b", "u", "w", "o", "e", "i", "n", "c", "s", "r", "h", "a","t", "v", ",", "j", "k", ";", ".", "q", "m", "\"", "z" ]
#Calculating and printing Fitness
fitness3=FitnessFunction(optimalKeyboard,weights, dataset2)
print("The most optimal keyboard has the following fitness: ",fitness3[1])
print("The keyboard has a Fitness/Character of:",(fitness3[1]/len(dataset2)))